In [2]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [3]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

In [4]:
RDS_HOST = 'banco-db.cfaunaiv0bu7.us-east-1.rds.amazonaws.com'
#RDS_HOST=config.get('DW', 'RDS_HOST')
#RDS_HOST
print(RDS_HOST)

banco-db.cfaunaiv0bu7.us-east-1.rds.amazonaws.com


In [5]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""  

##### Dimensión de escuela

In [90]:
sql_query = 'SELECT * FROM p_school;'
dim_school = pd.read_sql(sql_query, postgres_driver)
dim_school.head()

id_school school
0          1     GP
1          2     MS

##### Dimensión de genero

In [91]:
sql_query = 'SELECT * FROM p_sex;'
dim_sex = pd.read_sql(sql_query, postgres_driver)
dim_sex.head()

id_sex sexo
0       1    M
1       2    F

##### Dimensión de edad

In [92]:
sql_query = 'SELECT * FROM p_age;'
dim_age = pd.read_sql(sql_query, postgres_driver)
dim_age.head()

id_age age
0       1  15
1       2  16
2       3  17
3       4  18
4       5  19

#####  Dimensión de tipo de trabajo

In [93]:
sql_query = 'SELECT * FROM p_job;'
dim_job = pd.read_sql(sql_query, postgres_driver)
dim_job.head()

id_job  job_name relationship
0       1   at_home       father
1       2    health       father
2       3     other       father
3       4  services       father
4       5   teacher       father

#### Dimension de tutor

In [94]:
sql_query = 'SELECT * FROM p_guardian;'
dim_guardian = pd.read_sql(sql_query, postgres_driver)
dim_guardian.head()

id_guardian guardian
0            1   father
1            2   mother
2            3    other

##### Creamos dimensión de servicio de internet

In [95]:
sql_query = 'SELECT * FROM p_internet;'
dim_internet = pd.read_sql(sql_query, postgres_driver)
dim_internet.head()

id_internet internet
0            1      yes
1            2       no

##### Dimensión de situación sentimental

In [96]:
sql_query = 'SELECT * FROM p_romantic;'
dim_romantic = pd.read_sql(sql_query, postgres_driver)
dim_romantic.head()

id_romantic romantic
0            1       no
1            2      yes

#### Tabla de Hechos

In [97]:
sql_query = '''SELECT id_grades,  id_school,
                              id_sex, id_age, id_job, id_guardian, id_internet, id_romantic,
                              G1, G2, G3, subject FROM P_grades;'''
dim_grades = pd.read_sql(sql_query, postgres_driver)
dim_grades.head()

id_grades  id_school  id_sex  id_age  id_job  id_guardian  id_internet  \
0          0          2       1       5       4            1            2   
1          1          2       1       7       8            3            1   
2          2          1       1       2       3            3            1   
3          3          1       1       2      10            1            1   
4          4          2       2       5       3            1            1   

   id_romantic  g1  g2  g3 subject  
0            1   5   1   5       1  
1            1  13   9   1       1  
2            1   9   4  12       1  
3            2   8  10  14       1  
4            2   3   1   5       0

##### Lectura de Archivo desde S3

In [81]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [82]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)

hdmch-16002200


In [83]:
S3_BUCKET_NAME = 'hdmch-16002200'

In [84]:
#extraemos todo lo que está en el bucket
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)

remoteFileList

['Rango de edades.xlsx', 'T.xlsx']

##### leemos data de S3

In [87]:
import io

for remoteFile in remoteFileList:
    try:
        file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
        data = file['Body'].read()
        df_calificaciones = pd.read_excel(io.BytesIO(data), engine='openpyx1')
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

df_calificaciones.head()

No es un archivo.
Unknown engine: openpyx1
No es un archivo.
Unknown engine: openpyx1


school  sex  age  job  guardian  internet  romantic  G1  G2  G3  \
0       1    2    4    1         2         2         1   5   6   6   
1       1    2    3    1         1         1         1   5   5   6   
2       1    2    1    1         2         1         1   7   8  10   
3       1    2    1    2         2         1         2  15  14  15   
4       1    2    2    3         1         2         1   6  10  10   

       Materia  
0  Matemáticas  
1  Matemáticas  
2  Matemáticas  
3  Matemáticas  
4  Matemáticas

#### 1) Ralizamos Calculos de rendimiento del estudiante

#### 1.1 Cálculo del promedio por estudiante

In [19]:

FactorPromedioXEstudiante =  [(df_calificaciones['G1'] + df_calificaciones['G2']+ df_calificaciones['G3'])/3]
FactorPromedioXEstudiante

[0        5.666667
 1        5.333333
 2        8.333333
 3       14.666667
 4        8.666667
           ...    
 1039    10.333333
 1040    15.333333
 1041    10.666667
 1042    10.000000
 1043    10.666667
 Length: 1044, dtype: float64]

In [20]:
df = df_calificaciones.apply(lambda row: (row['G1'] + row['G2'] + row['G3'])/3,  axis=1)
PAlumno = pd.DataFrame({"Promedio":df})
PAlumno

Promedio
0      5.666667
1      5.333333
2      8.333333
3     14.666667
4      8.666667
...         ...
1039  10.333333
1040  15.333333
1041  10.666667
1042  10.000000
1043  10.666667

[1044 rows x 1 columns]

In [21]:
df = df_calificaciones.apply(lambda row: (row['G1'] + row['G2'] + row['G3'])/3,  axis=1) 
PAlumno2 = pd.DataFrame({"Promedio":df})
PAlumno2

Promedio
0      5.666667
1      5.333333
2      8.333333
3     14.666667
4      8.666667
...         ...
1039  10.333333
1040  15.333333
1041  10.666667
1042  10.000000
1043  10.666667

[1044 rows x 1 columns]

#### 1.2 Cálculo del promedio de clase por trimestre

In [22]:

PG1= df_calificaciones.iloc[:, 7].mean()
PG2= df_calificaciones.iloc[:, 8].mean()
PG3= df_calificaciones.iloc[:, 9].mean()

PromedioGeneral = (PG1+PG2+PG3)/3
PromedioGeneral

11.267241379310343

In [23]:
PAlumno2["Rendimiento"] = PAlumno2["Promedio"].apply (lambda x: 'Aprobado' if x > PromedioGeneral else 'Reprobado')
PAlumno2

Promedio Rendimiento
0      5.666667   Reprobado
1      5.333333   Reprobado
2      8.333333   Reprobado
3     14.666667    Aprobado
4      8.666667   Reprobado
...         ...         ...
1039  10.333333   Reprobado
1040  15.333333    Aprobado
1041  10.666667   Reprobado
1042  10.000000   Reprobado
1043  10.666667   Reprobado

[1044 rows x 2 columns]

In [24]:
Materia = pd.DataFrame({"Materia":df_calificaciones.Materia})
Materia

Materia
0     Matemáticas
1     Matemáticas
2     Matemáticas
3     Matemáticas
4     Matemáticas
...           ...
1039       Idioma
1040       Idioma
1041       Idioma
1042       Idioma
1043       Idioma

[1044 rows x 1 columns]

In [25]:
c =  pd.DataFrame(pd.concat([PAlumno2['Promedio'],PAlumno2['Rendimiento'], Materia['Materia']], axis=1))
c

Promedio Rendimiento      Materia
0      5.666667   Reprobado  Matemáticas
1      5.333333   Reprobado  Matemáticas
2      8.333333   Reprobado  Matemáticas
3     14.666667    Aprobado  Matemáticas
4      8.666667   Reprobado  Matemáticas
...         ...         ...          ...
1039  10.333333   Reprobado       Idioma
1040  15.333333    Aprobado       Idioma
1041  10.666667   Reprobado       Idioma
1042  10.000000   Reprobado       Idioma
1043  10.666667   Reprobado       Idioma

[1044 rows x 3 columns]

### 1- Rendimiento de los alumnos por materia es el siguiente:

In [26]:
AlumnosAprobadosMatematicas = c[(c.Rendimiento == "Aprobado") & (c.Materia == "Matemáticas") ]
AlumnosAprobadosMatematicas.Rendimiento.count()


179

In [27]:
AlumnosReprobadosMatematicas = c[(c.Rendimiento == "Reprobado") & (c.Materia == "Matemáticas") ]
AlumnosReprobadosMatematicas.Rendimiento.count()

216

In [28]:
AlumnosAprobadosIdioma = c[(c.Rendimiento == "Aprobado") & (c.Materia == "Idioma") ]
AlumnosAprobadosIdioma.Rendimiento.count()

353

In [29]:
AlumnosReprobadosIdioma = c[(c.Rendimiento == "Reprobado") & (c.Materia == "Idioma") ]
AlumnosReprobadosIdioma.Rendimiento.count()

296

#### El rendimiento de los Alumnos de la materia de matematica es:

In [30]:
RendimientoPositivo=  AlumnosAprobadosMatematicas.Rendimiento.count()/c.Rendimiento.count()
RendimientoPositivo * 100

17.1455938697318

#### El rendimiento de los Alumnos de la materia de Idioma es:

In [31]:
RendimientoPositivo2=  AlumnosAprobadosIdioma.Rendimiento.count()/c.Rendimiento.count()
RendimientoPositivo2 * 100

33.81226053639847

R/ Los estudiantes de idioma tiene mayor aprobación que los de matemáticas, sin embargo, todavia la mayoria de los alumnos pierden ambas materias

### 2- Rendimiento de los alumnos por genero:

In [32]:
d =  pd.DataFrame(pd.concat([PAlumno2['Promedio'],PAlumno2['Rendimiento'], df_calificaciones['sex'], df_calificaciones['age'],df_calificaciones['job']
                             ,df_calificaciones['guardian'],df_calificaciones['romantic'],df_calificaciones['internet']], axis=1))
d

Promedio Rendimiento  sex  age  job  guardian  romantic  internet
0      5.666667   Reprobado    2    4    1         2         1         2
1      5.333333   Reprobado    2    3    1         1         1         1
2      8.333333   Reprobado    2    1    1         2         1         1
3     14.666667    Aprobado    2    1    2         2         2         1
4      8.666667   Reprobado    2    2    3         1         1         2
...         ...         ...  ...  ...  ...       ...       ...       ...
1039  10.333333   Reprobado    2    5    4         2         1         1
1040  15.333333    Aprobado    2    4    5         2         1         1
1041  10.666667   Reprobado    2    4    3         2         1         2
1042  10.000000   Reprobado    1    3    4         2         1         1
1043  10.666667   Reprobado    1    4    4         2         1         1

[1044 rows x 8 columns]

In [33]:
AlumnosAprobadosMasculinos = d[(d.Rendimiento == "Aprobado") & (d.sex == 1) ]
AlumnosAprobadosMasculinos.Rendimiento.count()

218

In [34]:
AlumnosAprobadosFemeninos = d[(d.Rendimiento == "Aprobado") & (d.sex == 2) ]
AlumnosAprobadosFemeninos.Rendimiento.count()

314

#### El rendimiento de estudiantes masculinos:

In [35]:
RendimientoPositivoMasculinos=  AlumnosAprobadosMasculinos.Rendimiento.count()/c.Rendimiento.count()
RendimientoPositivoMasculinos * 100

20.88122605363985

#### El rendimiento de los Alumnos Femeninos:

In [36]:
RendimientoPositivoFemenino = AlumnosAprobadosFemeninos.Rendimiento.count() /c.Rendimiento.count()
RendimientoPositivoFemenino * 100

30.07662835249042

R/ los estudiantes femeninos tiene mejor desempñeo que los masuculinos

### 3- Rendimiento de los estudiantes según el tipo de trabajo que tiene el padre

In [39]:
RendimientoEstPadreTrabajaHogar = d[(d.Rendimiento == "Aprobado") & (d.job == 1) ]
RendimientoEstPadreTrabajaHogar.Rendimiento.count()


66

In [41]:
RendimientoEstPadreTrabajaSalud = d[(d.Rendimiento == "Aprobado") & (d.job == 2) ]
RendimientoEstPadreTrabajaSalud.Rendimiento.count()

53

In [45]:
RendimientoEstPadreTrabajaOtros = d[(d.Rendimiento == "Aprobado") & (d.job == 3) ]
RendimientoEstPadreTrabajaOtros.Rendimiento.count()

201

In [46]:
RendimientoEstPadreTrabajaServicios = d[(d.Rendimiento == "Aprobado") & (d.job == 4) ]
RendimientoEstPadreTrabajaServicios.Rendimiento.count()

136

In [47]:
RendimientoEstPadreTrabajaEnseñanza = d[(d.Rendimiento == "Aprobado") & (d.job == 5) ]
RendimientoEstPadreTrabajaEnseñanza.Rendimiento.count()

76

#### Rendimiento de los estudiantes cuyo padre trabaja en categoría "otros"

In [55]:
RendimientoEstPadreTrabajaOtros.Rendimiento.count() / d[(d.Rendimiento == "Aprobado")].Rendimiento.count() * 100

37.78195488721804

R/ se constáto que los estudiantes que tienen mejor rendimiento son los que cuyo padre trabaja en "otros" tipo de trabajo ya que abarcan el 37% de cinco tipos de trabajo

### 4- Rendimiento de los estudiantes según el tipo de tutor que tiene

In [51]:
RendiEstTutorPadre = d[(d.Rendimiento == "Aprobado") & (d.guardian == 1) ]
RendiEstTutorPadre.Rendimiento.count()

135

In [52]:
RendiEstTutorMadre = d[(d.Rendimiento == "Aprobado") & (d.guardian == 2)]
RendiEstTutorMadre.Rendimiento.count()

371

In [53]:
RendiEstTutorOtro = d[(d.Rendimiento == "Aprobado") & (d.guardian == 3)]
RendiEstTutorOtro.Rendimiento.count()

26

In [56]:
RendiEstTutorMadre.Rendimiento.count() / d[(d.Rendimiento == "Aprobado")].Rendimiento.count() * 100

69.73684210526315

R/ Se identificó que los estudiantes que tienen mayor rendimeinto son los que tienen un tutor madre, ya que el 69.73 % de los aprobados, tienen este tipo de tutor

### 4- Rendimiento de los estudiantes según si tiene o no una relación sentimental

In [57]:
RendiEstSirelacion = d[(d.Rendimiento == "Aprobado") & (d.romantic == 2) ]
RendiEstSirelacion.Rendimiento.count()

183

In [58]:
RendiEstNorelacion = d[(d.Rendimiento == "Aprobado") & (d.romantic == 1) ]
RendiEstNorelacion.Rendimiento.count()

349

In [59]:
RendiEstNorelacion.Rendimiento.count() / d[(d.Rendimiento == "Aprobado")].Rendimiento.count() * 100

65.6015037593985

R/ se identificó que los estudiantes que tienen mejor desempeño que el promedio, son los que No tienen alguna relación sentimiental

### 5- Rendimiento de los estudiantes según si tienen acceso internet o no

In [60]:
RendiEstSiAccesoInt = d[(d.Rendimiento == "Aprobado") & (d.internet == 1) ]
RendiEstSiAccesoInt.Rendimiento.count()

444

In [61]:
RendiEstNoAccesoInt = d[(d.Rendimiento == "Aprobado") & (d.internet == 2) ]
RendiEstNoAccesoInt.Rendimiento.count()

88

In [62]:
RendiEstSiAccesoInt.Rendimiento.count() / d[(d.Rendimiento == "Aprobado")].Rendimiento.count() * 100

83.45864661654136

R/ se identificó que el 83.45 % que tienen acceso a internet son los que tienen mejor rendimiento

##### Creación de Instancias en AWS para DW

In [63]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

In [64]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')
print(aws_conn)

In [65]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['banco-db', 'dw-db']


In [66]:
rdsIdentifier = 'dw-db'

In [67]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS_MYSQL', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="mysql",
            MasterUsername=config.get('RDS_MYSQL', 'DB_USER'),
            MasterUserPassword=config.get('RDS_MYSQL', 'DB_PASSWORD'),
            Port=int(config.get('RDS_MYSQL', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


##### Antes de conectarnos recordemos esperar el tiempo suficiente para agregar la nueva instancia y agregar el puerto 3306 al grupo de seguridad.

In [68]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_DW_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_DW_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

dw-db.cfaunaiv0bu7.us-east-1.rds.amazonaws.com


In [69]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


##### Recordemos instalar MySQLdb

In [77]:
import mysql.connector as mysqlC
myDw = mysqlC.connect(
host=RDS_DW_HOST, 
user=config.get('RDS_MYSQL', 'DB_USER'),
password=config.get('RDS_MYSQL', 'DB_PASSWORD')
)
mycursor = myDw.cursor()
mycursor.close()

True

In [79]:
import dw_proyecto
import mysql.connector as mysqlC
try:
    myDw = mysqlC.connect(
    host=RDS_DW_HOST, 
    user=config.get('RDS_MYSQL', 'DB_USER'),
    password=config.get('RDS_MYSQL', 'DB_PASSWORD'),
    database=config.get('RDS_MYSQL', 'DB_NAME')
    )

    mycursor = myDw.cursor()
    mycursor.execute(dw_proyecto.CREATE_DW, multi=True)
    myDw.commit()
    print("Data Warehouse Creado Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

ERROR: Error al crear la base de datos.
module 'dw_proyecto' has no attribute 'CREATE_DW'


##### Insertamos Data

In [80]:
mysql_driver = f"""mysql+pymysql://{config.get('RDS_MYSQL', 'DB_USER')}:{config.get('RDS_MYSQL', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_MYSQL', 'DB_PORT')}/{config.get('RDS_MYSQL', 'DB_NAME')}"""  

In [ ]:
#insertamos las escuelas.
dim_school.to_sql('dim_school', mysql_driver, index=False, if_exists='append')

In [ ]:
#insertamos los generos
dim_sex.to_sql('dim_sex', mysql_driver, index=False, if_exists='append')

505

In [ ]:
#Insertamos edad
dim_age.to_sql('dim_age', mysql_driver, 
                  index=False, if_exists='append', method='multi')

3

In [ ]:
# Insertamos el tipo de trabajo
dim_job.to_sql('dim_job', mysql_driver, 
                   index=False, if_exists='append', method='multi')

22

In [ ]:
# Insertamos el tipo de tutor
dim_guardian.to_sql('dim_guardian', mysql_driver, 
                    index=False, if_exists='append', method='multi')

107

In [ ]:
# Insertamos si el estudiante tien si o no Internet
dim_internet.to_sql('dim_internet', 
                            mysql_driver, index=False, if_exists='append', method='multi')

In [ ]:
# Insertamos la realción sentimental del alumno si tiene o no.
dim_romantic.to_sql('dim_romantic', 
                            mysql_driver, index=False, if_exists='append', method='multi')

In [ ]:
# Insertamos las calificaciones de los estudiantes
dim_grades.to_sql('dim_grades', 
                            mysql_driver, index=False, if_exists='append', method='multi')